In [12]:
import time

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import unit_cube
from ngsolve.krylovspace import CGSolver

In [13]:
n = specialcf.normal(3)
def norm(u, Mesh):
    with TaskManager():
        return sqrt(Integrate( InnerProduct(u,u) , Mesh))
def Curl(u):
    if u.dim == 3:
        return CF( (u[2].Diff(y)- u[1].Diff(z), u[0].Diff(z)- u[2].Diff(x), u[1].Diff(x)- u[0].Diff(y)) )
    if u.dim == 9:
        return CF( (Curl(u[0,:]),Curl(u[1,:]),Curl(u[2,:])),dims=(3,3) )
def Inc(u):
    return Curl((Curl(u)).trans)
P_n = OuterProduct(n,n) 
Q_n = Id(3) - OuterProduct(n,n) 
C_n = CF(      (0,n[2],-n[1],-n[2],0,n[0],n[1],-n[0],0), dims=(3,3) )
def C(V): 
    return CF( (0,V[2],-V[1],-V[2],0,V[0],V[1],-V[0],0), dims=(3,3) ) 


### Direct problem:
Given $\gamma$ let's find $$f=inc(\gamma)$$

In [14]:
def DirectProblem(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (x-0.5)**2 + (y-0.5)**2 +(z-0.5)**2))

        # our gamma
        PEAK = CF ( (peak, 0, 0 , 0,peak,0,0,0,peak), dims=(3,3))
        # true inc(gamma)
        IncPeak = CF( Inc(PEAK), dims=(3,3) )

        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( PEAK, bonus_intorder=10, dual=True)

        gfCurlCurlG = GridFunction(fesHCurlCurl)
        gfCurlCurlG.Set(IncPeak, BND,  bonus_intorder=9, dual=True)

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        u,v = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,v[0,:]),Cross(n,v[1,:]),Cross(n,v[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl, symmetric=True)
        a += InnerProduct(u,v)*dx 

        # linear form induced by the metric gfG
        f = LinearForm(fesHCurlCurl)

        # thetrahedron inc part
        f += InnerProduct(gfG.Operator("inc"),v)*dx        
        # faces part:
        f += ( InnerProduct(Q_n*n_cross_v, curl(gfG).trans) + Cross(gfG*n,n)*(curl(v)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        f += (gfG[n1,e]*v[e,t1] - gfG[n2,e]*v[e,t2])*dx(element_vb=BBND)
        
        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfCurlCurlG.vec
        inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
        gfCurlCurlG.vec.data += inverse * r
        #gfCurlCurlG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r

        print("error of my inc")
        Error = norm(IncPeak-gfCurlCurlG,mesh)
        print("Error inc(PEAK)-gfcurlCurlG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfCurlCurlG-IncPeak,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (gfCurlCurlG,mesh, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (IncPeak, mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        return Error

In [15]:
Maxh = 0.15
Order = 3
with TaskManager():
    DirectProblem(Maxh, Order)

CG NOT converged in 1000 iterations to residual 0.00010066656871742073
error of my inc
Error inc(PEAK)-gfcurlCurlG when order is 3-->0.11411750395541903


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

### Inverse problem:
Given $f$ let's find $\gamma$ such that $$inc(\gamma)=f$$
with all the conditions descrbed at the beginning of the notebook

Since we need to start from a function that is in the range of inc we need to take 
$$
\begin{pmatrix}
0 & peak & 0 \\
peak & 0 & 0 \\
0 & 0 & 0 \\
\end{pmatrix}
$$
peak is a function only in $z$ therefore since the $div\, div = 0$ it is in the range of $inc$

In [7]:

def S(V): 
    return V - (V.Trace())*Id(3)
def J(V): 
    return V - (V.Trace())/2*Id(3)

def InverseProblem(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():
        ##########################################################
        #           Coefficient and grid functions               #
        ##########################################################
           
        # continuous functions
        peak = exp(-25*( (z-0.5)**2))

        # this is the function that we need to find
        PEAK = CF ( (0, peak, 0 , peak,0,0,0,0,0), dims=(3,3))
        IncPEAK = CF ( Inc(PEAK), dims=(3,3))
        # grid functions 
        gfG = GridFunction(fesHCurlCurl) 
        gfG.Set ( IncPEAK, bonus_intorder=10, dual=True)

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        uc,vc = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,vc[0,:]),Cross(n,vc[1,:]),Cross(n,vc[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        # thetrahedron inc part
        a += InnerProduct(uc.Operator("inc"),vc)*dx        
        # faces part:
        a += ( InnerProduct(Q_n*n_cross_v, curl(uc).trans) + Cross(uc*n,n)*(curl(vc)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        a += (uc[n1,e]*vc[e,t1] - uc[n2,e]*vc[e,t2])*dx(element_vb=BBND)
        
        # regularization:
        #print(vc.Operators())
        #a += InnerProduct(J(vc),vd.Operator("symgrad")) *dx
        #a += InnerProduct(J(uc),ud.Operator("symgrad")) *dx

        f = LinearForm(fesHCurlCurl)
        f += InnerProduct(IncPEAK,vc)*dx

        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        f.Assemble()

        r = f.vec.CreateVector()
        r.data = f.vec - a.mat * gfG.vec
        #inverse = CGSolver(a.mat, pre.mat , printrates='\r', maxiter=1000,tol=1e-9)
        gfG.vec.data += a.mat.Inverse() * r
        #gfG.vec.data += a.mat.Inverse(freedofs=fesHCurlCurl.FreeDofs(),inverse="sparsecholesky") * r

        print("error of my inverted inc problem")
        Error = norm(PEAK-gfG,mesh)
        print(norm(PEAK-gfG,mesh))
        print(norm(gfG,mesh))
        print(norm(PEAK,mesh))
        print("Error PEAK-gfG when order is "+str(Order)+"-->"+str(Error) )
        Draw (gfG[1,0]-PEAK[1,0],mesh, draw_surf=False, clipping=(0,1,1), deformation=False)
        Draw (gfG[1,0],mesh)#, draw_surf=True, clipping=(0,1,1), deformation=False)
        Draw (PEAK[1,0], mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        Draw (Inc(PEAK)[1,0], mesh, clipping=(0,1,1),name = "Real inc gfG",deformation=False) 
        return Error

In [8]:
Maxh = 0.15
Order = 3
with TaskManager():
    InverseProblem(Maxh, Order)


error of my inverted inc problem
2.0979739193896426e+17
2.0979739193896426e+17
0.7080477210030366
Error PEAK-gfG when order is 3-->2.0979739193896432e+17


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…

## EigenValues
we need to solve $ Inc(u) = \lambda u$ The discretization leads to $ Au = \lambda Mu$

In [9]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import unit_square
import math
import scipy.linalg
from scipy import random
def S(V): 
    return V - (V.Trace())*Id(3)
def J(V): 
    return V - (V.Trace())/2*Id(3)

def EigenVals(Maxh,Order):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=Maxh))
    fesHCurlCurl = HCurlCurl(mesh, order=Order, dirichlet= ".*")
    with TaskManager():        
        # grid functions 
        u = GridFunction(fesHCurlCurl) 

        ##########################################################
        #              Linear and Bilinear forms                 #
        ##########################################################

        uc,vc = fesHCurlCurl.TnT()
        #some geometrical objects we need
        n_cross_v = CF( (Cross(n,vc[0,:]),Cross(n,vc[1,:]),Cross(n,vc[2,:])), dims=(3,3) )
        t1 =specialcf.EdgeFaceTangentialVectors(3)[:,0]
        t2 =specialcf.EdgeFaceTangentialVectors(3)[:,1]
        e = specialcf.tangential(3,True)
        n1 = Cross( t1, e)
        n2 = Cross( t2, e) 

        # Mass matrix
        a = BilinearForm(fesHCurlCurl)
        # thetrahedron inc part
        a += InnerProduct(uc.Operator("inc"),vc)*dx        
        # faces part:
        a += ( InnerProduct(Q_n*n_cross_v, curl(uc).trans) + Cross(uc*n,n)*(curl(vc)*n) )*dx(element_boundary=True)
        # Edges components: t'*v*C_n*n
        a += (uc[n1,e]*vc[e,t1] - uc[n2,e]*vc[e,t2])*dx(element_vb=BBND)
        
        # regularization:
        #print(vc.Operators())
        #a += InnerProduct(J(vc),vd.Operator("symgrad")) *dx
        #a += InnerProduct(J(uc),ud.Operator("symgrad")) *dx

        m = BilinearForm(fesHCurlCurl)
        m += InnerProduct(uc,vc)*dx

        ##########################################################
        #           Assemble, compute and print!                 #
        ##########################################################
        pre = Preconditioner(a, "local")
        a.Assemble()
        m.Assemble()

        r = u.vec.CreateVector()
        w = u.vec.CreateVector()
        Mu = u.vec.CreateVector()
        Au = u.vec.CreateVector()
        Mw = u.vec.CreateVector()
        Aw = u.vec.CreateVector()
        r.FV().NumPy()[:] = random.rand(fesHCurlCurl.ndof)
        u.vec.data = Projector(fesHCurlCurl.FreeDofs(), True) * r
        for i in range(20):
            Au.data = a.mat * u.vec
            Mu.data = m.mat * u.vec
            auu = InnerProduct(Au, u.vec)
            muu = InnerProduct(Mu, u.vec)
            # Rayleigh quotient
            lam = auu/muu
            print (lam / (math.pi**2))
            # residual
            r.data = Au - lam * Mu
            w.data = pre.mat * r.data
            w.data = 1/Norm(w) * w
            Aw.data = a.mat * w
            Mw.data = m.mat * w
        
            # setup and solve 2x2 small eigenvalue problem
            asmall = Matrix(2,2)
            asmall[0,0] = auu
            asmall[0,1] = asmall[1,0] = InnerProduct(Au, w)
            asmall[1,1] = InnerProduct(Aw, w)
            msmall = Matrix(2,2)
            msmall[0,0] = muu
            msmall[0,1] = msmall[1,0] = InnerProduct(Mu, w)
            msmall[1,1] = InnerProduct(Mw, w)
            # print ("asmall =", asmall, ", msmall = ", msmall)
        
        
            eval,evec = scipy.linalg.eigh(a=asmall, b=msmall)
            # print (eval, evec)
            u.vec.data = float(evec[0,0]) * u.vec + float(evec[1,0]) * w
            
        Draw (u, clipping=(0,0,1))


In [10]:
Maxh = 0.15
Order = 3
with TaskManager():
    EigenVals(Maxh, Order)


0.0468410223996691
-0.09762878377666756
-0.099400828698142
-0.10110297795555753
-0.10272869733947011
-0.10426992660608332
-0.10572058979117224
-0.10707098406645107
-0.10831481978804408
-0.10943928332838211
-0.11043738268440483
-0.11128881604545733
-0.11198683286410564
-0.1124938336658829
-0.11281437810372964
-0.11290168575493853
-0.11290311614279983
-0.11290453126423268
-0.11290591936669897
-0.11290729286064966


WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 3, 'order2d': 2, 'order3d': 2, 'draw_vol': True…